# 0.0 Imports

In [34]:
import pandas as pd
import inflection
import math

## 0.1 Funções auxiliares

# 1.0 Carregando os dados

In [15]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory= False)
df_store_raw = pd.read_csv('data/store.csv', low_memory= False)


#merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

# 2.0 Descrição dos dados

In [16]:
df1 = df_raw.copy()

## 2.1 Renomear colunas

In [27]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday',
            'SchoolHoliday', 'StoreType', 'Assortment','CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list( map(snakecase, cols_old) )

#renomear
df1.columns = cols_new

## 2.2 Dimensão dos dados

In [30]:
print( 'Number of Rows:{}'.format( df1.shape[0]))
print( 'Number of Cols:{}'.format( df1.shape[1]))

Number of Rows:1017209
Number of Cols:18


## 2.3 Tipo de dados

In [31]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [32]:
df1['date'] = pd.to_datetime( df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 2.4 verificar os NA

In [33]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 2.5 preencher os NA

In [38]:
df1['competition_distance'].max()

200000.0

In [49]:
#competition_distance              2642
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan ( x ) else x )

#competition_open_since_month    323348
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month']) else
                                                x['competition_open_since_month'], axis =1)

#competition_open_since_year     323348
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_year']) else
                                                x['competition_open_since_year'], axis =1)
#promo2                               0
#promo2_since_week               508031
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_week']) else
                                                x['promo2_since_week'], axis =1)
#promo2_since_year               508031
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_year']) else
                                                x['promo2_since_year'], axis =1)

#promo_interval                  508031
month_map = {1:'Jan',2:'fev',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
#substituir NA por 0
df1['promo_interval'].fillna(0, inplace=True)
#transaformando data em mes
df1['month_map'] = df1['date'].dt.month.map(month_map)
#verificando se a loja fez parte da promocao
df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 
                                                            if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

NameError: name 'dt1' is not defined

In [67]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

In [71]:
df1.sample(10).T

,624820,477383,645154,424970,167749,481869,18297,22246,761845,254011
store,91,949,355,941,500,975,458,1062,1086,960
day_of_week,3,3,6,1,2,6,3,7,7,6
date,2013-12-18 00:00:00,2014-04-30 00:00:00,2013-11-30 00:00:00,2014-06-16 00:00:00,2015-03-03 00:00:00,2014-04-26 00:00:00,2015-07-15 00:00:00,2015-07-12 00:00:00,2013-08-18 00:00:00,2014-12-13 00:00:00
sales,10615,7940,11285,7192,5800,5234,8539,0,0,7147
customers,1059,732,1155,662,382,551,736,0,0,568
open,1,1,1,1,1,1,1,0,0,1
promo,1,1,0,1,1,0,1,0,0,0
state_holiday,0,0,0,0,0,0,0,0,0,0
school_holiday,0,0,0,1,0,0,0,0,0,0
store_type,c,a,a,a,d,a,c,d,a,d
